# Model evaluation notebook

in this notebook we will evaluate % availability. definitions are:

X = 
    location: site_id
    time: hour, dow, weekend, is_holiday, 
    weather: temperature, dew point, relative humidity, wind direction, wind speed
    grid: net demand, solar, wind
y = % available (for site)

this will work for all sites.

How will we do train test split?
random 80-20 for now 


In [1]:
import os
if 'models' == os.getcwd().split('/')[-1]: os.chdir('..')
if 'ev_charging' == os.getcwd().split('/')[-1]: print('in the right place!')
else: os.chdir('/Users/varunvenkatesh/Documents/Github/ev_charging')
os.getcwd()

in the right place!


'/home/denny/Documents/mids/ev_charging'

In [2]:
import pandas as pd
from src.weather import get_processed_hourly_7day_weather


In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

from datetime import datetime

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

In [4]:
# hist_weather = pd.read_csv('./data/caltech_historical_weather')

In [5]:
weather_features = ['temperature_degC', 'dewpoint_degC','relative_humidity_%', 'wind_speed_mph']
hist_weather = pd.read_csv('./data/caltech_historical_weather.csv').set_index('time')[weather_features]

In [6]:
hist_weather

,temperature_degC,dewpoint_degC,relative_humidity_%,wind_speed_mph
time,,,,
2018-01-01 00:00:00,17.1,9.0,59.0,5.59
2018-01-01 01:00:00,17.3,9.5,60.0,5.84
2018-01-01 02:00:00,15.7,10.0,69.0,3.36
2018-01-01 03:00:00,14.5,11.3,81.0,4.72
2018-01-01 04:00:00,13.4,11.8,90.0,3.36
...,...,...,...,...
2021-09-14 19:00:00,26.8,10.1,35.0,5.84
2021-09-14 20:00:00,29.0,9.2,29.0,5.84
2021-09-14 21:00:00,31.2,5.6,20.0,8.08


In [7]:
from src.weather import get_processed_hourly_7day_weather
site2latlon = {'Caltech':(34.134785646454844, -118.11691382579643),
               'Office001':(37.33680466796926, -121.90743423142634),
               'JPL':(34.20142342818471, -118.17126565774107)}
future_weather = get_processed_hourly_7day_weather(*site2latlon['Caltech'])

https://api.weather.gov/points/34.134785646454844,-118.11691382579643
status code: 200
https://api.weather.gov/gridpoints/LOX/160,48/forecast/hourly


In [8]:
future_weather.to_csv('data/test_future_weather.csv')

 confirm that past and future weather have the same features

In [9]:
print(f'future weather features are \n{list(future_weather[weather_features].columns)}, \nand historical weather features are \n{list(hist_weather[weather_features].columns)}')

future weather features are 
['temperature_degC', 'dewpoint_degC', 'relative_humidity_%', 'wind_speed_mph'], 
and historical weather features are 
['temperature_degC', 'dewpoint_degC', 'relative_humidity_%', 'wind_speed_mph']


# get caiso data

In [10]:
from src import oasis
import datetime
import pandas as pd
from src.oasis import SystemDemand
import urllib
sd = SystemDemand()


def get_historical_caiso():
    start = datetime(2018,1,1)
    end = datetime(2020,3,1)
    
    demand_forecast = sd.get_demand_forecast(start, end)
    wind_solar_forecast = sd.get_wind_and_solar_forecast(start, end)
    wind_solar_forecast['INTERVALSTARTTIME_GMT'] = pd.to_datetime(wind_solar_forecast['INTERVALSTARTTIME_GMT'],
                                                              utc=True)
    solar_df = wind_solar_forecast[wind_solar_forecast['RENEWABLE_TYPE'] == 'Solar']
    wind_df = wind_solar_forecast[wind_solar_forecast['RENEWABLE_TYPE'] == 'Wind']
    
    return demand_forecast, solar_df, wind_df

In [4]:
start = datetime.datetime(2018,1,1)
end = datetime.datetime(2020,3,1)

demand_forecast = sd.get_demand_forecast(start, end)
wind_solar_forecast = sd.get_wind_and_solar_forecast(start, end)
wind_solar_forecast['INTERVALSTARTTIME_GMT'] = pd.to_datetime(wind_solar_forecast['INTERVALSTARTTIME_GMT'],
                                                          utc=True)
solar_df = wind_solar_forecast[wind_solar_forecast['RENEWABLE_TYPE'] == 'Solar']
wind_df = wind_solar_forecast[wind_solar_forecast['RENEWABLE_TYPE'] == 'Wind']

NoDataAvailableError: No data available for this query.

In [5]:
# set the date range to extract files
d1 = datetime.datetime(2022,1,1)
d2 = datetime.datetime(2022,2,1) # dt.date.today()
days = [d1 + datetime.timedelta(days=x) for x in range((d2-d1).days + 1)]

In [11]:
file_types = ['demand', 'netdemand', 'fuelsource', 'storage', 'co2', 'rtm_forecast']
url_format = "http://www.caiso.com/outlook/SP/History/%s/%s.csv"

In [23]:
day = days[0]
file_type = file_types[2]
url = url_format % (day.strftime("%Y%m%d"), file_type)
url

'http://www.caiso.com/outlook/SP/History/20220101/fuelsource.csv'

In [24]:
file = urllib.request.urlopen(url)

In [25]:
df = pd.read_csv(file)
df

,Time,Solar,Wind,Geothermal,Biomass,Biogas,Small hydro,Coal,Nuclear,Natural Gas,Large Hydro,Batteries,Imports,Other
0,00:00,-32,3829,882,281,205,146,20,2264,7280,1200,-116,8200,0
1,00:05,-32,3854,880,279,208,144,20,2265,7119,1129,117,8079,0
2,00:10,-32,3934,880,276,209,144,20,2264,6904,1114,149,8085,0
3,00:15,-32,4007,879,273,210,145,21,2265,6870,1135,164,7962,0
4,00:20,-32,3998,879,275,211,144,20,2265,6860,1102,152,7973,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,23:35,-42,1134,879,292,206,194,18,2265,9880,1321,-18,7483,0
284,23:40,-43,1108,880,292,206,195,18,2264,9907,1306,-77,7400,0
285,23:45,-43,1082,880,294,207,195,18,2265,9983,1297,-130,7384,0
286,23:50,-43,1079,879,291,207,194,17,2265,9961,1291,-57,7218,0


# Get historical demand

the data from the request is the bytes for a zip file. to get it to a pandas dataframe,
1. read the content with the io.BytesIO
2. open the bytes as a zip file with ZipFile
3. open the zip file by getting the first (and only) name in the extraction
4. read the data using the .read() method of the file object
5. convert the raw data to a string with BytesIO again
6. read the bytes to a dataframe with the pd.read_csv file

In [102]:
import pytz
from io import BytesIO
from zipfile import ZipFile

def _get_UTC_string(
    dt: datetime,
    local_tz: str = "America/Los_Angeles",
    fmt: str = "%Y%m%dT%H:%M-0000",
) -> str:
    """Convert local datetime to UTC string

    Converts datetime.datetime or pandas.Timestamp in local time to
    to UTC string for constructing HTTP request

    Args:
        dt (datetime.datetime): datetime to convert
        local_tz (str): timezone

    Returns:
        utc (str): UTC string
    """

    tz_ = pytz.timezone(local_tz)
    return tz_.localize(dt).astimezone(pytz.UTC).strftime(fmt)


In [111]:
url = 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&startdatetime={}&enddatetime={}'
start_date = datetime.datetime(2018,1,1) # make a datetime
end_date = start_date + datetime.timedelta(days=31)
print(_get_UTC_string(start_date), _get_UTC_string(end_date))
url.format(start_date, end_date)
def make_demand_url(start_date, end_date):
    start = _get_UTC_string(start_date)
    end = _get_UTC_string(end_date)
    base_url = 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&startdatetime={}&enddatetime={}'
    return base_url.format(start, end)

def caiso_call_for_df(url):
    
    # https://stackoverflow.com/questions/5710867/downloading-and-unzipping-a-zip-file-without-writing-to-disk
    r = requests.get(url)
    with ZipFile(BytesIO(r.content)) as zipedfolder:
        with zipedfolder.open(zipedfolder.namelist()[0]) as file:
            bytes_payload = file.read()

    raw = pd.read_csv(BytesIO(bytes_payload))
    
    TACS = ['PGE','SCE-TAC', 'PGE-TAC', 'CA ISO-TAC']
    MARKET_RUN_IDS = ['ACTUAL', 'DAM']
    
    filtered_df = raw[(raw['MARKET_RUN_ID'].isin(MARKET_RUN_IDS)) & (raw['TAC_AREA_NAME'].isin(TACS))]
    return filtered_df

20180101T08:00-0000 20180201T08:00-0000


In [113]:
print('starting process')
start_date = datetime.datetime(2018,1,1)
all_df = pd.DataFrame({})
while start_date < datetime.datetime(2020,3,1):
    end_date = start_date + datetime.timedelta(days=31)
    url = make_demand_url(start_date, end_date)
    print(url)
    df = caiso_call_for_df(url)
    
    if all_df.empty:
        all_df = df
    else:
        all_df = pd.concat([all_df, df], axis=0)
    start_date = end_date
print(start_date)
print(url)

starting process
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&startdatetime=20180101T08:00-0000&enddatetime=20180201T08:00-0000
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&startdatetime=20180201T08:00-0000&enddatetime=20180304T08:00-0000
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&startdatetime=20180304T08:00-0000&enddatetime=20180404T07:00-0000
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&startdatetime=20180404T07:00-0000&enddatetime=20180505T07:00-0000
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&startdatetime=20180505T07:00-0000&enddatetime=20180605T07:00-0000
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&startdatetime=20180605T07:00-0000&enddatetime=20180706T07:00-0000
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_

In [118]:
all_df['OPR_DT'].min()
all_df.to_csv('historical_caiso_actual_load.csv')


end

In [98]:

import requests
url = 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&startdatetime=20180101T08:00-0000&enddatetime=20180131T08:00-0000'
r = requests.get(url)
# https://stackoverflow.com/questions/5710867/downloading-and-unzipping-a-zip-file-without-writing-to-disk
with ZipFile(BytesIO(r.content)) as zipedfolder:
    with zipedfolder.open(zipedfolder.namelist()[0]) as file:
        test = file.read()
        # print(test)
from io import StringIO
TACS = ['PGE','SCE-TAC', 'PGE-TAC', 'CA ISO-TAC']
MARKET_RUN_IDS = ['ACTUAL', 'DAM']
raw = pd.read_csv(BytesIO(test))

In [99]:
# filters
raw[(raw['MARKET_RUN_ID'].isin(MARKET_RUN_IDS)) & (raw['TAC_AREA_NAME'].isin(TACS))]

,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,LOAD_TYPE,OPR_DT,OPR_HR,OPR_INTERVAL,MARKET_RUN_ID,TAC_AREA_NAME,LABEL,XML_DATA_ITEM,POS,MW,EXECUTION_TYPE,GROUP
552,2018-01-01T15:00:00-00:00,2018-01-01T16:00:00-00:00,0,2018-01-01,8,0,ACTUAL,CA ISO-TAC,Total Actual Hourly Integrated Load,SYS_FCST_ACT_MW,3.0,20661.0,ACTUAL,24
553,2018-01-01T21:00:00-00:00,2018-01-01T22:00:00-00:00,0,2018-01-01,14,0,ACTUAL,CA ISO-TAC,Total Actual Hourly Integrated Load,SYS_FCST_ACT_MW,3.0,20476.0,ACTUAL,24
554,2018-01-02T06:00:00-00:00,2018-01-02T07:00:00-00:00,0,2018-01-01,23,0,ACTUAL,CA ISO-TAC,Total Actual Hourly Integrated Load,SYS_FCST_ACT_MW,3.0,22584.0,ACTUAL,24
555,2018-01-01T09:00:00-00:00,2018-01-01T10:00:00-00:00,0,2018-01-01,2,0,ACTUAL,CA ISO-TAC,Total Actual Hourly Integrated Load,SYS_FCST_ACT_MW,3.0,20559.0,ACTUAL,24
556,2018-01-01T12:00:00-00:00,2018-01-01T13:00:00-00:00,0,2018-01-01,5,0,ACTUAL,CA ISO-TAC,Total Actual Hourly Integrated Load,SYS_FCST_ACT_MW,3.0,19613.0,ACTUAL,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153903,2018-01-31T03:00:00-00:00,2018-01-31T04:00:00-00:00,1,2018-01-30,20,0,DAM,SCE-TAC,Demand Forecast Day Ahead,SYS_FCST_DA_MW,2.2,12951.0,DAM,1956
153904,2018-01-30T21:00:00-00:00,2018-01-30T22:00:00-00:00,1,2018-01-30,14,0,DAM,SCE-TAC,Demand Forecast Day Ahead,SYS_FCST_DA_MW,2.2,12093.0,DAM,1956
153905,2018-01-30T14:00:00-00:00,2018-01-30T15:00:00-00:00,1,2018-01-30,7,0,DAM,SCE-TAC,Demand Forecast Day Ahead,SYS_FCST_DA_MW,2.2,10966.0,DAM,1956
153906,2018-01-30T08:00:00-00:00,2018-01-30T09:00:00-00:00,1,2018-01-30,1,0,DAM,SCE-TAC,Demand Forecast Day Ahead,SYS_FCST_DA_MW,2.2,9465.0,DAM,1956
